<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="ITM">
</head>

<img src="https://maua.br/images/selo-60-anos-maua.svg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->



# **P1 - RA PAR**

In [1]:
#@markdown Nome completo e RA (*\<nome\>, \<RA\>*)
Aluno = 'Vitor Soller, 21.01444-2' #@param {type:"string"}






In [2]:
path = 'https://github.com/Rogerio-mack/work/raw/main/data/'

In [4]:
import numpy as np
import pandas as pd

# **Case**: Airbnb

Neste case, analise dados sobre imóveis alugados pela plataforma Airbnb em NYC no ano de 2019.


In [5]:
df = pd.read_csv(path + 'AB_NYC_2019.csv')
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365  
0               1.57                               1                 0  
1               1.46                               2               137  
2               3.97                               3               182  
3               0.91                               2               286  
4               0.43                               2                59

# Q1. Proprietários com mais imóveis na plataforma

Obtenha os 3 proprietários ("hosts") com mais imóveis sendo anunciados na plataforma, e a respectiva quantidade de imóveis anunciados.


In [9]:
df.groupby('host_name').agg({'id':'count'}).sort_values('id',ascending=False).head(3)

id
host_name       
Sonder (NYC)   7
Mark           7
David          6

# Q2A. Locatários profissionais $\times$ anunciantes com um único imóvel

Considere o preço médio dos imóveis. Qual a diferença do preço médio dos imóveis dos anunciantes com mais de um imóvel e dos anunciantes com um único imóvel anunciado na plataforma?

In [12]:
df_mean = df.groupby('host_name').agg({'id':'count','price':'mean'})

In [13]:
df_mean_mais_de_um_imovel = df_mean[df_mean.id > 1]

In [14]:
df_mean_mais_de_um_imovel.head()

id       price
host_name                
Aaron       3   76.666667
Adam        4  162.500000
Adeyemi     2   27.500000
Adriano     2   77.500000
Alan        2  123.000000

In [15]:
df_mean_1_imovel = df_mean[df_mean.id == 1]

In [16]:
df_mean_1_imovel.head()

id  price
host_name           
A.B.        1   99.0
Abby        1   80.0
Abena       1   47.0
Abigail     1  125.0
Abram       1   91.0

In [22]:
len(df.host_name.unique()) == len(df_mean_mais_de_um_imovel.index) + len(df_mean_1_imovel.index)

True

In [210]:
preco_mais_de_um_imovel = df_mean_mais_de_um_imovel.price.mean()
preco_mais_um_imovel = df_mean_1_imovel.price.mean()

In [212]:
preco_mais_um_imovel

np.float64(151.3764258555133)

In [213]:
preco_mais_de_um_imovel

np.float64(153.04477124183006)

In [211]:
print( f'Diferença de preços: USD {preco_mais_de_um_imovel - preco_mais_um_imovel:.2f}')

Diferença de preços: USD 1.67


# Q3A.  Manhattan *outliers*

Qual o percentual de imóveis de Manhattan com preços estatisticamente discrepantes (*outliers*) com relação aos preços de todos os imóveis?



In [27]:
def identify_outliers(data):
  Q1 = data.quantile(0.25)
  Q3 = data.quantile(0.75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR
  outliers = data[(data < lower_bound) | (data > upper_bound)].index.tolist()
  return outliers


In [31]:
outliers = identify_outliers(df[df.neighbourhood_group == 'Manhattan'].price)

In [41]:
outliers

[23,
 35,
 54,
 82,
 97,
 150,
 151,
 209,
 297,
 308,
 323,
 339,
 365,
 499,
 510,
 563,
 623,
 645,
 677,
 680,
 684,
 707,
 708,
 718,
 727,
 738,
 795]

In [43]:
df[df.neighbourhood_group == 'Manhattan'].price.describe()

count     368.000000
mean      196.157609
std       234.131794
min        29.000000
25%        92.500000
50%       150.000000
75%       214.000000
max      2850.000000
Name: price, dtype: float64

In [40]:
perc_Manhattan = len(outliers)/len(df[df.neighbourhood_group == 'Manhattan'].price)*100

print( f'Percentuais de outliers de preços, Manhattan: {perc_Manhattan:.2f} %')


Percentuais de outliers de preços, Manhattan: 7.34 %


# Q4A. Preços $\times$ Descrições

Considere as descrições dos anúncios em `df.name`. Crie uma coluna `df['word']` que contém 'beautiful','sunny','spacious' ou 'heart' de acordo com a presença dessa palavra no anúncio. Caso não tenha nenhuma das palavras, assinale 'others'. Ao final, verifique a quantidade e o preço médio dos imóveis para cada grupo dessas palavras nos anúncios.

Atenção, considere a ordem 'beautiful','sunny','spacious' ou 'heart' (isto é, se o texto contém duas ou mais palavras, por exemplo 'beautiful' e 'sunny', considere 'beautiful'). Considere também quaisquer combinações de maiúsculas/minúsculas nas palavras.

In [44]:
word = ['beautiful','sunny','spacious','heart']

In [64]:
for index, name in df.name.items():
  for w in word:
    if w in str(name).lower():
      df.loc[index,'word'] = w
      print(index)
      print(w)
      print(name)
      break
    df.loc[index,'word'] = 'others'

6
sunny
Large Sunny Luxe Prvt Room/Midtown
11
sunny
Sunny and spacious bedroom
16
sunny
Sunny West Village Dream
17
beautiful
Beautiful Brooklyn Brownstone
19
spacious
large spacious room
33
spacious
Spacious 1 BR Near the Water UES
36
beautiful
Beautiful 1BR in Prime West Village
37
heart
Artsy Fashion Pad in the Heart of Williamsburg
40
spacious
Gorgeous spacious. 2 subways close
47
spacious
Spacious one bedroom apartment
49
sunny
Quiet and Sunny studio in EV
50
spacious
Inspiring spacious loft space
58
beautiful
BEAUTIFUL ARTIST APT IN NOLITA w/ Private Backyard
59
beautiful
Beautiful Prewar UWS Apartment
60
beautiful
Apt #2: Beautiful 2BR/1BA Crown Hgt
65
heart
Cozy bedroom in the heart of Bushwick
67
sunny
Safe Sunny South Facing Near All
73
sunny
Sunny and Spacious 1 bedroom apt
74
sunny
Modern sunny one bed apt
78
sunny
Large Sunny Nicely Furnished Studio
80
sunny
Sunny private room in Gramercy
81
heart
Best Location NYC! Studio in heart of The Village
87
spacious
Spacious Bedro

In [63]:
df['word']

0      others
1      others
2      others
3      others
4      others
        ...  
799     sunny
800    others
801    others
802    others
803    others
Name: word, Length: 804, dtype: object

In [65]:
# Q4A. Qual a palavra que nos anúncios que apresenta o maior e menor preços médios? (valores truncados)

df.groupby('word').agg({'price':'mean'}).sort_values('price')

price
word                 
spacious   128.516667
sunny      137.674419
beautiful  137.926829
others     157.886218
heart      166.388889

In [66]:
# heart,166.38;spacious,128.51

# Q5. Enriquecendo os dados

Enriqueça os dados dos imóveis com os valores dos impostos e multas municipais. Cacule a média de impostos e multas de cada grupo de vizinhaça, `neighbourhood_group`. Note que existem imóveis sem impostos e sem multas, esses são imóveis isentos, mas devem ser considerados nos cálculos de média (por exemplo, você pode considerar esses imóveis com valor 0).

In [83]:
df_taxes = pd.read_csv(path + 'AB_NYC_2019_taxes.csv')
df_taxes

id   type   value
0     35907127  taxes  1315.0
1       804815  taxes     NaN
2     22328585  taxes   900.0
3     27876600  taxes   869.0
4     34976037  taxes  1629.0
...        ...    ...     ...
1523  34284316  fines     NaN
1524  20325804  fines     NaN
1525  25872986  fines     NaN
1526  19365218  fines   906.0
1527   4235843  fines     NaN

[1528 rows x 3 columns]

In [82]:
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365    word  
0               1.57                               1                 0  others  
1               1.46                               2               137  others  
2               3.97                               3               182  others  
3               0.91                               2               286  others  
4               0.43                               2                59  others

In [84]:
df_taxes.fillna(0,inplace=True)

In [86]:
df_taxes

id   type   value
0     35907127  taxes  1315.0
1       804815  taxes     0.0
2     22328585  taxes   900.0
3     27876600  taxes   869.0
4     34976037  taxes  1629.0
...        ...    ...     ...
1523  34284316  fines     0.0
1524  20325804  fines     0.0
1525  25872986  fines     0.0
1526  19365218  fines   906.0
1527   4235843  fines     0.0

[1528 rows x 3 columns]

In [90]:
df_taxes_pivot = df_taxes.pivot(index='id',columns='type',values='value')

In [91]:
df_taxes_pivot

type       fines   taxes
id                      
20913      796.0   789.0
25235        0.0  1779.0
76761     1935.0  1516.0
135465       0.0     0.0
219482       0.0     0.0
...          ...     ...
36413779     0.0     0.0
36421287     0.0   748.0
36439512     0.0     0.0
36452721     0.0  1028.0
36472171     0.0  1120.0

[764 rows x 2 columns]

In [92]:
df_taxes_pivot.reset_index(inplace=True)

In [93]:
df_taxes_pivot

type        id   fines   taxes
0        20913   796.0   789.0
1        25235     0.0  1779.0
2        76761  1935.0  1516.0
3       135465     0.0     0.0
4       219482     0.0     0.0
..         ...     ...     ...
759   36413779     0.0     0.0
760   36421287     0.0   748.0
761   36439512     0.0     0.0
762   36452721     0.0  1028.0
763   36472171     0.0  1120.0

[764 rows x 3 columns]

In [112]:
df_merge = df.merge(df_taxes_pivot,how='left',on='id')

In [113]:
df_merge

id                                               name    host_id  \
0       20913                 Charming 1 bed GR8 WBurg LOCATION!      79402   
1       25235                  Large 2 Bedroom Great for Groups!      87773   
2       76761     Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm     118971   
3      135465                Garden apartment close to Manhattan     663879   
4      169002                   Modern Space in Charming Pre-war     805344   
..        ...                                                ...        ...   
799  36413779     Sunny & spacious room in a 3 Br - Williamsburg    2449230   
800  36421287  Nice & Lovely room in LIC.5 stops to Times Square  203866020   
801  36439512                       The Bushwick Backhouse Loft.  180336853   
802  36452721  Massage Spa. Stay overnight. Authors Artist dr...  274079964   
803  36472171                      1 bedroom in sunlit apartment   99144947   

       host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0     Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1         Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2         Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3    Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4           Alec           Manhattan              Harlem  40.82411  -73.94934   
..           ...                 ...                 ...       ...        ...   
799       Itamar            Brooklyn        Williamsburg  40.71812  -73.95495   
800        Karma              Queens           Sunnyside  40.73799  -73.92749   
801         Dash            Brooklyn            Bushwick  40.70470  -73.92250   
802      Richard            Brooklyn      Sheepshead Bay  40.59866  -73.95661   
803       Brenda           Manhattan              Inwood  40.86845  -73.92449   

           room_type  price  minimum_nights  number_of_reviews last_review  \
0    Entire home/apt    100               5                168  2018-07-22   
1    Entire home/apt    125              90                162  2019-06-28   
2    Entire home/apt    169               2                398  2019-06-28   
3    Entire home/apt    170               3                 86  2019-06-20   
4       Private room     65               2                 41  2019-06-16   
..               ...    ...             ...                ...         ...   
799     Private room     80               5                  0         NaN   
800     Private room     60               1                  0         NaN   
801     Private room     70               2                  0         NaN   
802     Private room    800               1                  0         NaN   
803     Private room     80               1                  0         NaN   

     reviews_per_month  calculated_host_listings_count  availability_365  \
0                 1.57                               1                 0   
1                 1.46                               2               137   
2                 3.97                               3               182   
3                 0.91                               2               286   
4                 0.43                               2                59   
..                 ...                             ...               ...   
799                NaN                               1                38   
800                NaN                               1                34   
801                NaN                               1               255   
802                NaN                               1                23   
803                NaN                               1                79   

       word   fines   taxes  
0    others   796.0   789.0  
1    others     0.0  1779.0  
2    others  1935.0  1516.0  
3    others     0.0     0.0  
4    others     NaN     NaN  
..      ...     ...     ...  
799   sunny     0.0     0.0  

In [115]:
df_merge.fillna({'fines' : 0, 'taxes': 0})

id                                               name    host_id  \
0       20913                 Charming 1 bed GR8 WBurg LOCATION!      79402   
1       25235                  Large 2 Bedroom Great for Groups!      87773   
2       76761     Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm     118971   
3      135465                Garden apartment close to Manhattan     663879   
4      169002                   Modern Space in Charming Pre-war     805344   
..        ...                                                ...        ...   
799  36413779     Sunny & spacious room in a 3 Br - Williamsburg    2449230   
800  36421287  Nice & Lovely room in LIC.5 stops to Times Square  203866020   
801  36439512                       The Bushwick Backhouse Loft.  180336853   
802  36452721  Massage Spa. Stay overnight. Authors Artist dr...  274079964   
803  36472171                      1 bedroom in sunlit apartment   99144947   

       host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0     Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1         Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2         Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3    Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4           Alec           Manhattan              Harlem  40.82411  -73.94934   
..           ...                 ...                 ...       ...        ...   
799       Itamar            Brooklyn        Williamsburg  40.71812  -73.95495   
800        Karma              Queens           Sunnyside  40.73799  -73.92749   
801         Dash            Brooklyn            Bushwick  40.70470  -73.92250   
802      Richard            Brooklyn      Sheepshead Bay  40.59866  -73.95661   
803       Brenda           Manhattan              Inwood  40.86845  -73.92449   

           room_type  price  minimum_nights  number_of_reviews last_review  \
0    Entire home/apt    100               5                168  2018-07-22   
1    Entire home/apt    125              90                162  2019-06-28   
2    Entire home/apt    169               2                398  2019-06-28   
3    Entire home/apt    170               3                 86  2019-06-20   
4       Private room     65               2                 41  2019-06-16   
..               ...    ...             ...                ...         ...   
799     Private room     80               5                  0         NaN   
800     Private room     60               1                  0         NaN   
801     Private room     70               2                  0         NaN   
802     Private room    800               1                  0         NaN   
803     Private room     80               1                  0         NaN   

     reviews_per_month  calculated_host_listings_count  availability_365  \
0                 1.57                               1                 0   
1                 1.46                               2               137   
2                 3.97                               3               182   
3                 0.91                               2               286   
4                 0.43                               2                59   
..                 ...                             ...               ...   
799                NaN                               1                38   
800                NaN                               1                34   
801                NaN                               1               255   
802                NaN                               1                23   
803                NaN                               1                79   

       word   fines   taxes  
0    others   796.0   789.0  
1    others     0.0  1779.0  
2    others  1935.0  1516.0  
3    others     0.0     0.0  
4    others     0.0     0.0  
..      ...     ...     ...  
799   sunny     0.0     0.0  

In [118]:
df_merge_mean = df_merge.groupby('neighbourhood_group').agg({'taxes':'mean','fines':'mean'})

In [119]:
df_merge_mean

taxes       fines
neighbourhood_group                         
Bronx                 851.052632  126.473684
Brooklyn              858.276873  314.472313
Manhattan             888.844633  314.980226
Queens                804.166667  322.769231
Staten Island        1157.500000  581.333333

In [123]:
df_merge_mean.loc['Manhattan','taxes'].astype(int)

np.int64(888)

In [124]:
df_merge_mean.loc['Manhattan','fines'].astype(int)

np.int64(314)

# Q6. Quebras

Construa no `Pandas` um "relatório" (`df_report`) como abaixo. Os valores são ordenados `neighbourhood_group`,`neighbourhood`. Cada "página" contém apenas 30 linhas. A cada "quebra" de `neighbourhood_group` é gerada uma linha com preço médio do `neighbourhood_group` e, então, é iniciada uma nova página.

O item `line_id` é opcional mas deve auxiliá-lo nessa tarefa.

Ao final produza a soma dos preços das páginas abaixo...

In [188]:
df.head()

id                                            name  host_id  \
0   20913              Charming 1 bed GR8 WBurg LOCATION!    79402   
1   25235               Large 2 Bedroom Great for Groups!    87773   
2   76761  Eveland the Place to Stay & Enjoy a 5-⭐️ 2bdrm   118971   
3  135465             Garden apartment close to Manhattan   663879   
4  169002                Modern Space in Charming Pre-war   805344   

     host_name neighbourhood_group       neighbourhood  latitude  longitude  \
0   Christiana            Brooklyn        Williamsburg  40.70984  -73.95775   
1       Shelly            Brooklyn  Bedford-Stuyvesant  40.68373  -73.92377   
2       Evelyn            Brooklyn         South Slope  40.66552  -73.99019   
3  Christopher            Brooklyn         Fort Greene  40.68626  -73.97598   
4         Alec           Manhattan              Harlem  40.82411  -73.94934   

         room_type  price  minimum_nights  number_of_reviews last_review  \
0  Entire home/apt    100               5                168  2018-07-22   
1  Entire home/apt    125              90                162  2019-06-28   
2  Entire home/apt    169               2                398  2019-06-28   
3  Entire home/apt    170               3                 86  2019-06-20   
4     Private room     65               2                 41  2019-06-16   

   reviews_per_month  calculated_host_listings_count  availability_365    word  
0               1.57                               1                 0  others  
1               1.46                               2               137  others  
2               3.97                               3               182  others  
3               0.91                               2               286  others  
4               0.43                               2                59  others

In [214]:
columns = ['neighbourhood_group', 'neighbourhood', 'price']

In [215]:
df_middle_report = df[columns].sort_values(columns)

In [216]:
df_middle_report

neighbourhood_group      neighbourhood  price
442               Bronx            Belmont     29
769               Bronx  Claremont Village     67
353               Bronx          Concourse     40
625               Bronx           Edenwald     80
597               Bronx           Edenwald     90
..                  ...                ...    ...
495       Staten Island        South Beach     20
607       Staten Island        South Beach    100
560       Staten Island         St. George    130
714       Staten Island          Todt Hill    135
408       Staten Island      Tompkinsville     59

[804 rows x 3 columns]

In [217]:
df_middle_report.reset_index(inplace=True)

In [218]:
df_middle_report

index neighbourhood_group      neighbourhood  price
0      442               Bronx            Belmont     29
1      769               Bronx  Claremont Village     67
2      353               Bronx          Concourse     40
3      625               Bronx           Edenwald     80
4      597               Bronx           Edenwald     90
..     ...                 ...                ...    ...
799    495       Staten Island        South Beach     20
800    607       Staten Island        South Beach    100
801    560       Staten Island         St. George    130
802    714       Staten Island          Todt Hill    135
803    408       Staten Island      Tompkinsville     59

[804 rows x 4 columns]

In [219]:
df_middle_report.drop(columns=['index'],inplace=True)

In [220]:
df_mean_by_neighbourhood_group = df_middle_report.groupby('neighbourhood_group').agg({'price':'mean'})

In [221]:
df_mean_by_neighbourhood_group.head()

price
neighbourhood_group            
Bronx                 71.500000
Brooklyn             128.087879
Manhattan            196.157609
Queens                92.227848
Staten Island         89.428571

In [222]:
df_mean_by_neighbourhood_group.rename(columns={'price':'zzzzzzMEAN'},inplace=True)

In [223]:
df_mean_by_neighbourhood_group.reset_index(inplace=True)

In [224]:
df_mean_by_neighbourhood_group

neighbourhood_group  zzzzzzMEAN
0               Bronx   71.500000
1            Brooklyn  128.087879
2           Manhattan  196.157609
3              Queens   92.227848
4       Staten Island   89.428571

In [225]:
df_mean_by_neighbourhood_group_melt = df_mean_by_neighbourhood_group.melt(id_vars='neighbourhood_group',value_vars='zzzzzzMEAN')

In [226]:
df_mean_by_neighbourhood_group_melt.head()

neighbourhood_group    variable       value
0               Bronx  zzzzzzMEAN   71.500000
1            Brooklyn  zzzzzzMEAN  128.087879
2           Manhattan  zzzzzzMEAN  196.157609
3              Queens  zzzzzzMEAN   92.227848
4       Staten Island  zzzzzzMEAN   89.428571

In [227]:
df_mean_by_neighbourhood_group_melt.rename(columns={'value':'price', 'variable': 'neighbourhood'},inplace=True)

In [228]:
df_mean_by_neighbourhood_group_melt

neighbourhood_group neighbourhood       price
0               Bronx    zzzzzzMEAN   71.500000
1            Brooklyn    zzzzzzMEAN  128.087879
2           Manhattan    zzzzzzMEAN  196.157609
3              Queens    zzzzzzMEAN   92.227848
4       Staten Island    zzzzzzMEAN   89.428571

In [229]:
df_middle_report.head()

neighbourhood_group      neighbourhood  price
0               Bronx            Belmont     29
1               Bronx  Claremont Village     67
2               Bronx          Concourse     40
3               Bronx           Edenwald     80
4               Bronx           Edenwald     90

In [230]:
df_report_concat = pd.concat([df_mean_by_neighbourhood_group_melt,df_middle_report],axis=0)

In [231]:
df_report_concat.sort_values(columns,inplace=True)

In [232]:
df_report_concat.reset_index(inplace=True)

In [233]:
df_report_concat.drop(columns=['index'],inplace=True)

In [234]:
df_report_concat.iloc[-10::,:]

neighbourhood_group  neighbourhood       price
799              Queens       Woodside  120.000000
800              Queens     zzzzzzMEAN   92.227848
801       Staten Island  Arden Heights   83.000000
802       Staten Island  Randall Manor   99.000000
803       Staten Island    South Beach   20.000000
804       Staten Island    South Beach  100.000000
805       Staten Island     St. George  130.000000
806       Staten Island      Todt Hill  135.000000
807       Staten Island  Tompkinsville   59.000000
808       Staten Island     zzzzzzMEAN   89.428571

In [ ]:
sum_price_pages = df_report[ df_report.page_id.isin([1,24,25,28]) ].groupby('page_id').price.sum().astype(int).sum()

print(f'A soma dos preços das páginas 1, 24, 25 e 28 é: {sum_price_pages}')

#### Aqui um *sample* do report.

In [ ]:
df_report.iloc[-10::,:]

neighbourhood_group  neighbourhood       price  page_id  line_id
795              Queens       Woodside   75.000000       28       17
796              Queens       Woodside  120.000000       28       18
797              Queens           MEAN   92.227848       28       19
798       Staten Island  Randall Manor   99.000000       29        1
799       Staten Island    South Beach   20.000000       29        2
800       Staten Island    South Beach  100.000000       29        3
801       Staten Island     St. George  130.000000       29        4
802       Staten Island      Todt Hill  135.000000       29        5
803       Staten Island  Tompkinsville   59.000000       29        6
804       Staten Island           MEAN   89.428571       29        7

Q7. Considere as fases do processo CRISP DM. Relacione ao menos 6 tarefas presentes na preparação dos dados.Na etapa

Na etapa Entendimento do Negócio
-Entender sobre o que dados são


Na etapa Preparação de Dados
- Compreender os dados trabalhados para poder analisá-los, suas colunas, médias, outliers

Na etapa Praparação dos dados
- Preencher dados ausentes com valores, podem ser zerado, médias, etc

Na etapa de modelagem
- Entender a perguntar a se fazer sobre o assunto, encontrar a melhor maneira de achá-la

Na etapa avaliação
- Empregar correação do modelo, como por exemplo se o local influencia no preço médio, ou calcular o R2

Na etapa distribuição:
- Organizar os dados, gráficos, apresentações, ou até mesmo um notebook bem explicado. Responder as perguntar: Como e por quê




